In [2]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-09 21:32:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.30MB/s    in 0.3s    

2023-01-09 21:32:36 (3.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [8]:
# Get the number of rows in the DataFrame.
df.count()

341931

In [9]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url2 = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url2)

df2 = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [10]:
df2.count()

9002021

# Transform the Data

In [11]:
df.createOrReplaceTempView("software")

In [12]:
df.createOrReplaceTempView("wireless")

## Create the "review_id_table".

In [13]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = spark.sql("select review_id, customer_id, product_id, product_parent, review_date from software")
review_id_df.show()
review_id_df.printSchema()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
|R2C1DJSCC8UFS6|   37107850|B00EP7AP7C|     279360628| 2015-08-31|
|R1AXGS1W4YFXMX|     302120|B00OW2PET4|     729971168| 2015-08-31|
|R1XU1B93402SYJ|   20193077|B00N4OLCRO|     776572654| 2015-08-31|
|R2U432NB3OPVR0|   13106017|B005CELN8W|     222071424| 2015-08-31|
|R3R6FIMIOQ5SP9|   32587108|B005CELL1G|     168801430| 2015-08-31|
|R12TX6V09C9QNQ|   32020017|B00H09BOXQ|     455502995| 2015-08-31|
|R33UCII6YKUMKV|   13376158|B00MCLGAAO|     652069521| 2015-08

In [14]:
# Create the "review_id_df2" DataFrame with the appropriate columns and data types.
review_id_df2 = spark.sql("select review_id, customer_id, product_id, product_parent, review_date from wireless")
review_id_df2.show()
review_id_df2.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
|R2C1DJSCC8UFS6|   37107850|B00EP7AP7C|     279360628| 2015-08-31|
|R1AXGS1W4YFXMX|     302120|B00OW2PET4|     729971168| 2015-08-31|
|R1XU1B93402SYJ|   20193077|B00N4OLCRO|     776572654| 2015-08-31|
|R2U432NB3OPVR0|   13106017|B005CELN8W|     222071424| 2015-08-31|
|R3R6FIMIOQ5SP9|   32587108|B005CELL1G|     168801430| 2015-08-31|
|R12TX6V09C9QNQ|   32020017|B00H09BOXQ|     455502995| 2015-08-31|
|R33UCII6YKUMKV|   13376158|B00MCLGAAO|     652069521| 2015-08

## Create the "products" Table

In [15]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = spark.sql("select product_id, product_title from software")
products_df.show()
products_df.printSchema()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MUTIDKI|McAfee 2015 Inter...|
|B00EPACNUG|Hallmark Card Stu...|
|B00164AZA4| Search and Rescue 4|
|B00E6LIEFM|      Quickbooks Pro|
|B00VWEBG06|Windows 7 Profess...|
|B00EP7AP7C|Family Tree Maker...|
|B00OW2PET4|World of Farming:...|
|B00N4OLCRO|Photoshop Element...|
|B005CELN8W|Phantom EFX WMS S...|
|B005CELL1G|PaperPort Profess...|
|B00H09BOXQ|Windows 7 Profess...|
|B00MCLGAAO| QuickBooks Pro 2015|
|B00P6U8C5Y|Block Financial H...|
|B00MA5TPN6|Parallels Desktop...|
|B000EQ868I|Cosmi Print Perfe...|
|B00MYXTCMI|TurboTax Business...|
|B008PIQXP0|I.R.I.S. Readiris...|
|B00KAFOXXU|Finale PrintMusic...|
|B00OKSOLE4|Microsoft Office ...|
|B01019T6O0|Microsoft Windows...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [16]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = spark.sql("select product_id, product_title from wireless")
products_df.show()
products_df.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MUTIDKI|McAfee 2015 Inter...|
|B00EPACNUG|Hallmark Card Stu...|
|B00164AZA4| Search and Rescue 4|
|B00E6LIEFM|      Quickbooks Pro|
|B00VWEBG06|Windows 7 Profess...|
|B00EP7AP7C|Family Tree Maker...|
|B00OW2PET4|World of Farming:...|
|B00N4OLCRO|Photoshop Element...|
|B005CELN8W|Phantom EFX WMS S...|
|B005CELL1G|PaperPort Profess...|
|B00H09BOXQ|Windows 7 Profess...|
|B00MCLGAAO| QuickBooks Pro 2015|
|B00P6U8C5Y|Block Financial H...|
|B00MA5TPN6|Parallels Desktop...|
|B000EQ868I|Cosmi Print Perfe...|
|B00MYXTCMI|TurboTax Business...|
|B008PIQXP0|I.R.I.S. Readiris...|
|B00KAFOXXU|Finale PrintMusic...|
|B00OKSOLE4|Microsoft Office ...|
|B01019T6O0|Microsoft Windows...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## Create the "customers" Table

In [17]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = spark.sql("select customer_id,count(customer_id) as customer_count from software group by customer_id")
customers_df.show()
customers_df.printSchema()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15634680|             1|
|   44696507|             1|
|   19901367|             1|
|   11337682|             1|
|   52021773|             1|
|   29916198|             1|
|   42030944|             1|
|   52765209|             1|
|   13519509|             2|
|   39320350|             5|
|   23249582|             1|
|   46351928|             3|
|   44179620|             1|
|     637252|             1|
|   34038793|             1|
|   51136490|             1|
|   49354815|             1|
|     149761|             2|
|   43333798|             1|
|    4657027|             1|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [18]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df2 = spark.sql("select customer_id,count(customer_id) as customer_count from wireless group by customer_id")
customers_df2.show()
customers_df2.printSchema()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15634680|             1|
|   44696507|             1|
|   19901367|             1|
|   11337682|             1|
|   52021773|             1|
|   29916198|             1|
|   42030944|             1|
|   52765209|             1|
|   13519509|             2|
|   39320350|             5|
|   23249582|             1|
|   46351928|             3|
|   44179620|             1|
|     637252|             1|
|   34038793|             1|
|   51136490|             1|
|   49354815|             1|
|     149761|             2|
|   43333798|             1|
|    4657027|             1|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



## Create the "vine_table".

In [19]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df=spark.sql("select review_id, star_rating, helpful_votes, total_votes, vine from software")
vine_df.show()
vine_df.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|          1|            2|          2|   N|
|R12NR0R5A9F7FT|          5|            0|          0|   N|
|R1LSH74R9XAP59|          2|            0|          1|   N|
|R1QXUNTF76K7L6|          2|            0|          0|   N|
|R2F7DR75PS8NKT|          5|            0|          0|   N|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|
|R1XU1B93402SYJ|          1|            1|          1|   N|
|R2U432NB3OPVR0|          5|            0|          0|   N|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|
|R12TX6V09C9QNQ|          5|            0|          0|   N|
|R33UCII6YKUMKV|          3|            2|          2|   N|
| RZKDAB9TGO053|          1|            0|          0|   N|
|R2EMN2EEDN73ZA|          4|            

In [20]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df2=spark.sql("select review_id, star_rating, helpful_votes, total_votes, vine from wireless")
vine_df2.show()
vine_df2.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|          1|            2|          2|   N|
|R12NR0R5A9F7FT|          5|            0|          0|   N|
|R1LSH74R9XAP59|          2|            0|          1|   N|
|R1QXUNTF76K7L6|          2|            0|          0|   N|
|R2F7DR75PS8NKT|          5|            0|          0|   N|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|
|R1XU1B93402SYJ|          1|            1|          1|   N|
|R2U432NB3OPVR0|          5|            0|          0|   N|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|
|R12TX6V09C9QNQ|          5|            0|          0|   N|
|R33UCII6YKUMKV|          3|            2|          2|   N|
| RZKDAB9TGO053|          1|            0|          0|   N|
|R2EMN2EEDN73ZA|          4|            

# Load

In [24]:
mode = "append"
jdbc_url="jdbc:postgresql://bigdatachallenge.cmisjhtztxsg.us-east-2.rds.amazonaws.com:5432/bigdatachallenge"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [25]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url = jdbc_url, table = 'review_id_df', mode = mode, properties = config)
review_id_df2.write.jdbc(url = jdbc_url, table = 'review_id_df2', mode = mode, properties = config)


Py4JJavaError: ignored

In [26]:
# Write products_df to table in RDS
products_df.write.jdbc(url = jdbc_url, table = 'products_df',mode = mode, properties = config)
products_df2.write.jdbc(url = jdbc_url, table = 'products_df2',mode = mode, properties = config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url = jdbc_url, table = 'customers_df',mode = mode, properties = config)
customers_df2.write.jdbc(url = jdbc_url, table = 'customers_df2',mode = mode, properties = config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url = jdbc_url, table = 'vine_df',mode = mode, properties = config)
vine_df2.write.jdbc(url = jdbc_url, table = 'vine_df2',mode = mode, properties = config)